In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-12-24 15:30:20--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.232.181.106, 34.206.253.53, 34.206.9.96, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.232.181.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]   5.11M  9.64MB/s    in 0.5s    

2018-12-24 15:30:20 (9.64 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [0]:
LOG_DIR = './tb'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://63adc34e.ngrok.io


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving conv.py to conv.py
User uploaded file "conv.py" with length 2087 bytes


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 24 09:53:02 2018

@author: maheryatim
"""

import tensorflow as tf
import numpy as np
"""""""""
From OpenAI Baselines
"""""""""

def intprod(x):
    return int(np.prod(x))


def conv2d(x, num_filters, name, filter_size=(3, 3), stride=(1, 1), pad="SAME", dtype=tf.float32, collections=None,
           summary_tag=None):
    with tf.variable_scope(name):
        stride_shape = [1, stride[0], stride[1], 1]
        filter_shape = [filter_size[0], filter_size[1], int(x.get_shape()[3]), num_filters]

        fan_in = intprod(filter_shape[:3])
        
        fan_out = intprod(filter_shape[:2]) * num_filters
        
        w_bound = np.sqrt(6. / (fan_in + fan_out))

        w = tf.get_variable("W", filter_shape, dtype, tf.random_uniform_initializer(-w_bound, w_bound),
                            collections=collections)
        b = tf.get_variable("b", [1, 1, 1, num_filters], initializer=tf.zeros_initializer(),
                            collections=collections)

        if summary_tag is not None:
            tf.summary.image(summary_tag,
                             tf.transpose(tf.reshape(w, [filter_size[0], filter_size[1], -1, 1]),
                                          [2, 0, 1, 3]),
                             max_images=10)

        return tf.nn.conv2d(x, w, stride_shape, pad) + b
    

def conv_(inputs):
    
    x = tf.nn.relu(conv2d(inputs, 32, "l1", [8, 8], [4, 4], pad="VALID"))
    x = tf.nn.relu(conv2d(x, 64, "l2", [4, 4], [2, 2], pad="VALID"))
    #x = tf.nn.relu(conv2d(x, 64, "l3", [2, 2], [1, 1], pad="VALID"))
    x = tf.layers.batch_normalization(x)
   
    return x

def conv_to_rnn(inputs, batch, seq, tboard = False): 
    '''
    Convolution network followed by an lstm network
    
    '''
    
    x = conv_(inputs)           
    
    image = x[0:1, :, :, 0:16]
    image = tf.transpose(image, perm=[3,1,2,0])
    tf.summary.image('Image_output_conv1', image)
    
    
    s = x.get_shape().as_list()
    print(s)
    x = tf.reshape(x, [batch, seq, 5184])

    
    return x

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 24 11:12:36 2018

@author: maheryatim
"""
import numpy as np 

class SumTree():
    """
    This SumTree code is modified version of Morvan Zhou: 
    https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/5.2_Prioritized_Replay_DQN/RL_brain.py
    """
    data_pointer = 0
     
    """
    Here we initialize the tree with all nodes = 0, and initialize the data with all values = 0
    """
    def __init__(self, capacity, timesteps):
        self.capacity = capacity # Number of leaf nodes (final nodes) that contains experiences
        
        self.timesteps = timesteps
        
        # Generate the tree with all nodes values = 0
        # To understand this calculation (2 * capacity - 1) look at the schema above
        # Remember we are in a binary node (each node has max 2 children) so 2x size of leaf (capacity) - 1 (root node)
        # Parent nodes = capacity - 1
        # Leaf nodes = capacity
        self.tree = np.zeros(2 * capacity - 1)
        self.i = 0
        """ tree:
            0
           / \
          0   0
         / \ / \
        0  0 0  0  [Size: capacity] it's at this line that there is the priorities score (aka pi)
        """
        
        # Contains the experiences (so the size of data is capacity)
        self.data = np.zeros((capacity,84*84*2+3), dtype=object)
    
    
    """
    Here we add our priority score in the sumtree leaf and add the experience in data
    """
    def add(self, priority, data):
        # Look at what index we want to put the experience
        tree_index = self.data_pointer + self.capacity - 1
        
        """ tree:
            0
           / \
          0   0
         / \ / \
tree_index  0 0  0  We fill the leaves from left to right
        """
                
        # Update data frame
        self.data[self.data_pointer] = data
        
        # Update the leaf
        self.update(tree_index, priority)
        
        # Add 1 to data_pointer
        self.data_pointer += 1
        
        if self.data_pointer >= self.capacity:  # If we're above the capacity, you go back to first index (we overwrite)
            self.data_pointer = 0
            
        
            
    """
    Update the leaf priority score and propagate the change through tree
    """
    def update(self, tree_index, priority):
        # Change = new priority score - former priority score
        change = priority - self.tree[tree_index]
        self.tree[tree_index] = priority
        
        # then propagate the change through tree
        while tree_index != 0:    # this method is faster than the recursive loop in the reference code
            
            """
            Here we want to access the line above
            THE NUMBERS IN THIS TREE ARE THE INDEXES NOT THE PRIORITY VALUES
            
                0
               / \
              1   2
             / \ / \
            3  4 5  [6] 
            
            If we are in leaf at index 6, we updated the priority score
            We need then to update index 2 node
            So tree_index = (tree_index - 1) // 2
            tree_index = (6-1)//2
            tree_index = 2 (because // round the result)
            """
            tree_index = (tree_index - 1) // 2
            self.tree[tree_index] += change
    
    
    """
    Here we get the leaf_index, priority value of that leaf and experience associated with that index
    """
    def get_leaf(self, v):
        """
        Tree structure and array storage:
        Tree index:
             0         -> storing priority sum
            / \
          1     2
         / \   / \
        3   4 5   6    -> storing priority for experiences
        Array type for storing:
        [0,1,2,3,4,5,6]
        """
        data_w_history = []
        parent_index = 0
        
        while True: # the while loop is faster than the method in the reference code
            left_child_index = 2 * parent_index + 1
            right_child_index = left_child_index + 1
            
            # If we reach bottom, end the search
            if left_child_index >= len(self.tree):
                leaf_index = parent_index
                break
            
            else: # downward search, always search for a higher priority node
                
                if v <= self.tree[left_child_index]:
                    parent_index = left_child_index
                    
                else:
                    v -= self.tree[left_child_index]
                    parent_index = right_child_index
            
        data_index = leaf_index - self.capacity + 1
        
        if len(self.data[(data_index - self.timesteps)  : data_index +1]) == self.timesteps  +1 :
           
            data_w_history.append(self.data[(data_index - self.timesteps) : data_index + 1])
            
        else :
           ref =  self.data[: data_index + 1]
           pad = np.zeros_like(self.data[len(ref) : self.timesteps +1])
           
           
           assert len(np.concatenate((pad, self.data[: data_index + 1]))) == self.timesteps + 1
           #print(np.shape(np.concatenate((pad, self.data[: data_index + 1]))))
           data_w_history.append(np.concatenate((pad, self.data[: data_index + 1]), axis = 0))
          
        return leaf_index, self.tree[leaf_index], data_w_history
    
    @property
    def total_priority(self):
        return self.tree[0] # Returns the root node



class Memory(object):  # stored as ( s, a, r, s_ ) in SumTree
    """
    This Memory class is modified based on the original code from:
    https://github.com/jaara/AI-blog/blob/master/Seaquest-DDQN-PER.py
    """
    epsilon = 0.01  # small amount to avoid zero priority
    alpha = 0.6  # [0~1] convert the importance of TD error to priority
    beta = 0.4  # importance-sampling, from initial value increasing to 1
    beta_increment_per_sampling = 0.01
    abs_err_upper = 1.  # clipped abs error

    def __init__(self, capacity, timesteps):
        self.tree = SumTree(capacity, timesteps)

    def store(self, transition):
        max_p = np.max(self.tree.tree[-self.tree.capacity:])
        if max_p == 0:
            max_p = self.abs_err_upper
        self.tree.add(max_p, transition)   # set the max p for new p

    def sample(self, n):
        
        b_idx = []
        b_memory = []
        ISWeights = []
        
        #b_idx, b_memory, ISWeights = np.empty((n,), dtype=np.int32), np.empty((n,1)), np.empty((n, 1))
        
        pri_seg = self.tree.total_priority / n       # priority segment
        self.beta = np.min([1., self.beta + self.beta_increment_per_sampling])  # max = 1

        min_prob = np.min(self.tree.tree[-self.tree.capacity:]) / self.tree.total_priority     # for later calculate ISweight
        for i in range(n):
            a, b = pri_seg * i, pri_seg * (i + 1)
            v = np.random.uniform(a, b)
            idx, p, data = self.tree.get_leaf(v)
            prob = p / self.tree.total_priority
            ISWeights.append(np.power(prob/(min_prob + 1.e-5), -self.beta))
            
            b_idx.append(idx)
            
            b_memory.append(data)
        return b_idx, b_memory, ISWeights

    def batch_update(self, tree_idx, abs_errors):
        abs_errors += self.epsilon  # convert to abs and avoid 0
        clipped_errors = np.minimum(abs_errors, self.abs_err_upper)
        ps = np.power(clipped_errors, self.alpha)
        
        for ti, p in zip(tree_idx, ps):
            self.tree.update(ti, p)


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 24 09:56:58 2018

@author: maheryatim
"""


import functools
import tensorflow as tf
import warnings
"""
from conv import conv_to_rnn
"""
warnings.filterwarnings('ignore')


mapping = {}

def register(name):
    def _thunk(f):
        mapping[name] = f
        return f
    return _thunk


def get_network_builder(name):
    """
    If you want to register your own network outside models.py, you just need:
    Usage Example:
    -------------
    from baselines.common.models import register
    @register("your_network_name")
    def your_network_define(**net_kwargs):
        ...
        return network_fn
    """
    if name in mapping:
        return mapping[name]
        
    else:
        raise ValueError('Unknown network type: {}'.format(name))

def huber_loss(x, delta=1.0):
    """Reference: https://en.wikipedia.org/wiki/Huber_loss"""
    return tf.where(
        tf.abs(x) < delta,
        tf.square(x) * 0.5,
        delta * (tf.abs(x) - 0.5 * delta)
    )

       
def doublewrap(function):
    """
    A decorator decorator, allowing to use the decorator to be used without
    parentheses if no arguments are provided. All arguments must be optional.
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator


@doublewrap
def define_scope(function, scope=None, *args, **kwargs):
    """
    A decorator for functions that define TensorFlow operations. The wrapped
    function will only be executed once. Subsequent calls to it will directly
    return the result so that operations are added to the graph only once.
    The operations added by the function live within a tf.variable_scope(). If
    this decorator is used with arguments, they will be forwarded to the
    variable scope. The scope name defaults to the name of the wrapped
    function.
    """
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)
    return decorator


@register('lstm')
class Model:

    def __init__(self, inputs, params, seq, batch, action_dim, rewards = None, actions = None, \
                 actions_next = None, q_tar = None, w = None, double = True, dueling = True):
        
        self.gamma = 0.9
        self.actions_dim = action_dim
        self.inputs = inputs
        self.double = double
        self.actions_next = actions_next
        self.params = params
        self.dueling = dueling 
        self.seq = seq
        self.batch = batch
        self.ISWeights = w
        self.rewards = rewards
        self.actions = actions
        self.q_tar = q_tar
        self.lr = 1.e-4
        self.prediction
        self.optimize 
        self.error
        
     
        
    @define_scope(initializer=tf.contrib.slim.xavier_initializer())
    def prediction(self):
        #x = tf.layers.batch_normalization(self.inputs)
        
        x = conv_to_rnn(self.inputs, self.batch, self.seq)
        lstm_layers = [tf.nn.rnn_cell.LSTMCell(
                            size,
                            initializer=tf.contrib.layers.xavier_initializer(),
                            forget_bias=0) for size in self.params]
                        
        multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(lstm_layers)
        x, _ = tf.nn.dynamic_rnn(
                    cell=multi_rnn_cell, inputs=x, dtype=tf.float32)
        
        x = tf.layers.batch_normalization(x)
        x = tf.reshape(x, (self.batch, self.params[-1]*self.seq))
        
        
        if self.dueling : 
            advs = tf.contrib.layers.fully_connected(x, num_outputs=256, activation_fn=tf.nn.relu)
            advs = tf.contrib.layers.fully_connected(advs, num_outputs=self.actions_dim, activation_fn=None)
            state_value = tf.contrib.layers.fully_connected(x, num_outputs=256, activation_fn=tf.nn.relu)
            state_value = tf.contrib.layers.fully_connected(state_value, num_outputs=1, activation_fn=None)
            action_scores_mean = tf.reduce_mean(advs, 1)
            action_scores_centered = advs - tf.expand_dims(action_scores_mean, 1)
            q_out = state_value + action_scores_centered
            out = q_out

        else:
            out = tf.contrib.layers.fully_connected(x, num_outputs=self.actions_dim, activation_fn=tf.nn.relu)
            out = tf.contrib.layers.fully_connected(out, num_outputs=self.actions_dim, activation_fn=tf.nn.relu)
            
        
        
        return out      
  
    
    
    @define_scope()
    def error(self):
                
                self.q_eval_wrt_a = tf.reduce_sum(self.prediction * tf.one_hot(self.actions, self.actions_dim), 1)
                if self.double : 
                    actions_index = self.actions_next + tf.range(
                                           0, self.batch) * self.q_tar.get_shape().as_list()[1]
                    pred_target = tf.gather(tf.reshape(self.q_tar, [-1]), actions_index)
                    TD = pred_target*0.9 + self.rewards 
                else : 
                    pred_target = self.q_tar
                    m = tf.reduce_max(pred_target, axis = 1)
                    TD = m *self.gamma + self.rewards 
                    
                TD = tf.stop_gradient(TD)
                error = TD - self.q_eval_wrt_a
                errors = huber_loss(error)
                self.loss = tf.reduce_mean(self.ISWeights * errors)
                return  self.loss
            
    @define_scope()
    def abs_error(self):
        TD = tf.reduce_max(self.q_tar, axis = 1)*self.gamma + self.rewards 
        self.abs_errors = tf.abs(TD-self.q_eval_wrt_a)
        return self.abs_errors
        
    @define_scope()
    def optimize(self):
                """
                grads_and_vars=tf.train.AdamOptimizer(self.lr).compute_gradients(self.error)
                for g, v in grads_and_vars:
                    if g is not None:
                        #print(format(v.name))
                        tf.summary.histogram("{}/grad_histogram".format(v.name), g)
                        tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                
                """  
                self._train_op = tf.train.AdamOptimizer(self.lr).minimize(self.error)
                return self._train_op


    
@register('lstm_target')
class Model_:

    def __init__(self, inputs, params, seq, batch, actions_dim, dueling = True):
        
        self.dueling = dueling 
        self.inputs = inputs
        self.params = params
        self.seq = seq
        self.batch = batch
        self.actions_dim = actions_dim
        self.prediction
        
    
    @define_scope(initializer=tf.contrib.slim.xavier_initializer())
    def prediction(self):
        #x = tf.layers.batch_normalization(self.inputs)      
        x = conv_to_rnn(self.inputs, self.batch, self.seq)
        lstm_layers = [tf.nn.rnn_cell.LSTMCell(
                            size,
                            initializer=tf.contrib.layers.xavier_initializer(),
                            forget_bias=0) for size in self.params]
                        
        multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(lstm_layers)
        x, _ = tf.nn.dynamic_rnn(
                    cell=multi_rnn_cell, inputs=x, dtype=tf.float32)
        x = tf.layers.batch_normalization(x)
        
        x = tf.reshape(x, (self.batch, self.params[-1]*self.seq))
        
        
        if self.dueling : 
            advs = tf.contrib.layers.fully_connected(x, num_outputs=256, activation_fn=tf.nn.relu)
            advs = tf.contrib.layers.fully_connected(advs, num_outputs=self.actions_dim, activation_fn=None)
            state_value = tf.contrib.layers.fully_connected(x, num_outputs=256, activation_fn=tf.nn.relu)
            state_value = tf.contrib.layers.fully_connected(state_value, num_outputs=1, activation_fn=None)
            action_scores_mean = tf.reduce_mean(advs, 1)
            action_scores_centered = advs - tf.expand_dims(action_scores_mean, 1)
            q_out = state_value + action_scores_centered
            out = q_out

        else:
            out = tf.contrib.layers.fully_connected(x, num_outputs=self.actions_dim, activation_fn=tf.nn.relu)
            out = tf.contrib.layers.fully_connected(out, num_outputs=self.actions_dim, activation_fn=tf.nn.relu)
            
        
        return out      

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 24 11:01:53 2018

@author: maheryatim
"""

import tensorflow as tf
import functools
import numpy as np 
#from utils import get_session 



def perturb(original_scope, perturbed_scope):

    filter_vars = ['fully_connected_1', 'fully_connected_2']
    original_params = []
    original_params_per = []
    for var in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=original_scope):
        for filter_var in filter_vars:
            if filter_var in var.name:
                original_params_per.append(var)
            else : 
                 original_params.append(var)
    perturbed_params = []
    perturbed_params_per = []

    for var in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=original_scope):
            for filter_var in filter_vars:
                if filter_var in var.name:
                    perturbed_params_per.append(var)
                else : 
                    perturbed_params.append(var)

    return original_params, original_params_per, perturbed_params, perturbed_params_per


def adapt(original_scope, perturbed_scope):

    filter_vars = ['fully_connected_1', 'fully_connected_2']
    original_params = []
    original_params_per = []
    for var in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=original_scope):
        for filter_var in filter_vars:
            if filter_var in var.name:
                original_params_per.append(var)
            else : 
                 original_params.append(var)
    perturbed_params = []
    perturbed_params_per = []

    for var in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=original_scope):
            for filter_var in filter_vars:
                if filter_var in var.name:
                    perturbed_params_per.append(var)
                else : 
                    perturbed_params.append(var)

    return original_params, original_params_per, perturbed_params, perturbed_params_per



def change(original_scope, perturbed_scope, param_noise_scale) : 
    
    if perturbed_scope == 'adaptive_model' : 
       original_params, original_params_per, perturbed_params, perturbed_params_per\
       = adapt(original_scope, perturbed_scope)
    elif perturbed_scope == 'perturbed_model' : 
        original_params, original_params_per, perturbed_params, perturbed_params_per\
        = perturb(original_scope, perturbed_scope)

               
    for var, perturbed_var in zip(original_params, perturbed_params):
                    tf.assign(perturbed_var, var)
    for var, perturbed_var in zip(original_params_per, perturbed_params_per):
                    tf.assign(perturbed_var, var + tf.random_normal(shape=tf.shape(var), mean=0., stddev=param_noise_scale))





       
def doublewrap(function):
    """
    A decorator decorator, allowing to use the decorator to be used without
    parentheses if no arguments are provided. All arguments must be optional.
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator


@doublewrap
def define_scope(function, scope=None, *args, **kwargs):
    """
    A decorator for functions that define TensorFlow operations. The wrapped
    function will only be executed once. Subsequent calls to it will directly
    return the result so that operations are added to the graph only once.
    The operations added by the function live within a tf.variable_scope(). If
    this decorator is used with arguments, they will be forwarded to the
    variable scope. The scope name defaults to the name of the wrapped
    function.
    """
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)
    return decorator


class noisy_actions : 
    
    def __init__(self, params, obs, step, delta, eps_state, q_value,q_values_adaptive,q_values_perturbed):
        
        self.timesteps_rnn = tf.placeholder(tf.int32, shape=(None), name='shape')
        self.batch_ = tf.placeholder(tf.int32, shape=(None), name='shape')
        self.states = tf.placeholder(tf.float32, shape=[None,84,84,1])
        self.sess = get_session()
        self.obs = obs
        self.step = step           
        self.delta = delta
        self.eps_state = eps_state
        self.q_values = q_value
        self.q_values_adaptive = q_values_adaptive
        self.q_values_perturbed = q_values_perturbed
        self.act_

        
    @define_scope
    def act_(self): 
        with tf.variable_scope("var", reuse=tf.AUTO_REUSE):
            variance = tf.get_variable("variance", (), initializer=tf.constant_initializer(2.19), trainable=False)
        
            
        #once epsilon is 0.01, stop updating params, perturbed = original barely 
        
        if self.step == 0 : 
            change('Q_value','perturbed_model', variance)
        
        if self.step % 100 == 0 and self.step > 0 : 
            change('Q_value','adaptive_model', variance)
            
            kl = tf.reduce_sum(tf.nn.sigmoid(self.q_values) * (tf.log(tf.nn.sigmoid(self.q_values))\
                                                 - tf.log(tf.nn.sigmoid(self.q_values_adaptive))),\
                           axis=-1)
            mean_kl = tf.reduce_mean(kl)
                
            if mean_kl < self.delta : 
                variance.assign(variance * 1.01)
            else : variance.assign(variance / 1.01)
        
        
        random_action = np.random.randint(0, 9)
        if np.random.uniform() < self.eps_state:
                action = random_action 
        else : action = tf.argmax(self.q_values_perturbed, axis=1)
                
        return action



In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 24 11:00:42 2018

@author: maheryatim
"""

import tensorflow as tf
import os  
import multiprocessing 
import joblib


def get_s(): 
    return tf.Session()

def get_session(config=None):
    """Get default session or create one with a given config"""
    
    sess = tf.get_default_session()
   
    if sess is None:
        #tf.reset_default_graph()
        
        sess = make_session(config=config, make_default=True)
    return sess

def make_session(config=None, num_cpu=None, make_default=False, graph=None):
    """Returns a session that will use <num_cpu> CPU's only"""
    if num_cpu is None:
        num_cpu = int(os.getenv('RCALL_NUM_CPU', multiprocessing.cpu_count()))
    if config is None:
        config = tf.ConfigProto(
            allow_soft_placement=True,
            inter_op_parallelism_threads=num_cpu,
            intra_op_parallelism_threads=num_cpu)
        config.gpu_options.allow_growth = True

    if make_default:
        return tf.InteractiveSession(config=config, graph=graph)
    else:
        return tf.Session(config=config, graph=graph)






ALREADY_INITIALIZED = set()

def initialize():
    """Initialize all the uninitialized variables in the global scope."""
    new_variables = set(tf.global_variables()) - ALREADY_INITIALIZED
    get_session().run(tf.variables_initializer(new_variables))
    ALREADY_INITIALIZED.update(new_variables)
    
    
    
def save_variables(save_path, variables=None, sess=None):
    sess = sess or get_session()
    variables = variables or tf.trainable_variables()

    ps = sess.run(variables)
    save_dict = {v.name: value for v, value in zip(variables, ps)}
    dirname = os.path.dirname(save_path)
    if any(dirname):
        os.makedirs(dirname, exist_ok=True)
    joblib.dump(save_dict, save_path)

def load_variables(load_path, variables=None, sess=None):
    sess = sess or get_session()
    variables = variables or tf.trainable_variables()

    loaded_params = joblib.load(os.path.expanduser(load_path))
    restores = []
    if isinstance(loaded_params, list):
        assert len(loaded_params) == len(variables), 'number of variables loaded mismatches len(variables)'
        for d, v in zip(loaded_params, variables):
            restores.append(v.assign(d))
    else:
        for v in variables:
            restores.append(v.assign(loaded_params[v.name]))

    sess.run(restores)


In [0]:




import tensorflow as tf
import numpy as np 
"""
from memory import Memory
from model import Model_, get_network_builder
from noisy_action import noisy_actions 
from utils import get_session, initialize, save_variables
"""
import os
import logging 
from PIL import Image



INPUT_SHAPE = (84, 84)



def process_observation(observation):
    assert observation.ndim == 3
    img = Image.fromarray(observation)
    img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
    processed_observation = np.array(img)
    assert processed_observation.shape == INPUT_SHAPE
    return processed_observation.astype('uint8')  # saves storage in experience memory

def process_state_batch(batch):
    processed_batch = batch.astype('float32') / 255.
    return processed_batch

def process_reward(reward):
    return np.clip(reward, -1., 1.)
act_dim = 9

class RDeepRL():

    def __init__(self, env,
                 model_type,
                 seed=None,
                 lr=5e-4,
                 history_size=2,
                 memory_size=5000,
                 exploration_fraction=0.1,
                 exploration_final_eps=0.02,
                 batch_size=100,
                 save_model_path=None,
                 steps_before_learning=1000,
                 gamma=0.9,
                 target_network_update_freq=1000,
                 load_path=None, 
                 double = False,
                 **network_kwargs):
        self.memory_size = memory_size
        self.double = double
        self.history_size = history_size
        self.memory = Memory(memory_size, self.history_size)
        self.model_type = model_type
        self.model_params = network_kwargs
        self.shape =  [84,84,1] 
        
        
        tf.reset_default_graph()

    def _build_graph(self):

        if self.model_type == 'lstm':
            model_Q = get_network_builder('lstm')
            model_target = get_network_builder('lstm_target')
            self.net_params = {
                
                'size': self.model_params.get('lstm_size', [256]),
                
            }

        elif self.model_type == 'rhh':
            model_Q = get_network_builder('lstm')
            model_target = get_network_builder('lstm_target')
            self.net_params = {
                """IMPLEMENT
                """
            }

        elif self.model_type == 'tensor':
            model_Q = get_network_builder('lstm')
            model_target = get_network_builder('lstm_target')
            self.net_params = {
                """IMPLEMENT
                """
            }

        else:
            raise NotImplementedError(
                "Unknown model type: '%s'" %
                self.model_type)

        self.timesteps_rnn = tf.placeholder(tf.int32, shape=(None), name='shape')
        self.batch_ = tf.placeholder(tf.int32, shape=(None), name='shape')
        
        with tf.variable_scope('Q_value', reuse = tf.AUTO_REUSE):
            self.states = tf.placeholder(
                tf.float32,
                shape=[
                    None,
                    self.shape[0],
                    self.shape[1],
                    self.shape[2]])
            self.actions_next = tf.placeholder(tf.int32, shape=(None), name='action_next')
            self.actions = tf.placeholder(tf.int32, shape=(None,), name='action')
            self.rewards = tf.placeholder(tf.float32, shape=(None,), name='reward')
            self.q_tar = tf.placeholder(
                tf.float32, [
                    None, env.action_space.n], name='Q_target')
            self.up_weights = tf.placeholder(
                tf.float32, shape=(
                    None,), name='priority_weights')
            self.model_Q = model_Q(
                self.states, self.net_params['size'], self.history_size +1, self.batch_, env.action_space.n,
                                            
                                             self.rewards,
                                             self.actions,
                                             self.actions_next,
                                             self.q_tar,
                                             self.up_weights)


        with tf.variable_scope('Target', reuse = tf.AUTO_REUSE):
            
            self.states_next = tf.placeholder(
                tf.float32,
                shape=[
                    None,
                    self.shape[0],
                    self.shape[1],
                    self.shape[2]])
            
             
            self.model_target = model_target(self.states_next,
                                             self.net_params['size'],
                                             self.history_size +1,
                                             self.batch_,
                                             env.action_space.n,
                                             )
        
        for var in tf.trainable_variables():
            tf.summary.histogram(var.name, var) 
        
        
        self.obs = tf.placeholder(tf.float32,shape=[None,84,84,1])
        self.step = tf.placeholder(tf.int32)             
        self.delta = tf.placeholder(tf.float32)
        
        self.q_value = tf.placeholder(tf.float32, [None, act_dim ])
        self.q_values_adaptive = tf.placeholder(tf.float32, [None, act_dim ])
        self.q_values_perturbed = tf.placeholder(tf.float32, [None, act_dim ])
        with tf.variable_scope('perturbed_model'):
          
            self.m_perturbed = Model_(self.states, self.net_params['size'], self.history_size +1, self.batch_ , act_dim )
            
        with tf.variable_scope('adaptive_model'):
          
            self.m_adapt = Model_(self.states,self.net_params['size'], self.history_size +1, self.batch_ , act_dim )
        
        self.action_ = noisy_actions(self.net_params['size'],self.obs, self.step,  self.delta, 0.01,
                                     self.q_value, self.q_values_adaptive,self.q_values_perturbed)
        self.eps_rand = 0.1
        
     
        self.sess = get_session()
        initialize()
     
        
    def update_target_weights(self, scope_q, scope_target):

        with tf.variable_scope('soft_replacement'):
           
        
            e_params = tf.trainable_variables(
                    
                    scope=scope_target)
            t_params = tf.trainable_variables(
                    scope=scope_q)[
                    :len(e_params)]
        
        for i in range(len(e_params)):
            self.sess.run(e_params[i].assign(tf.multiply(t_params[i], 0.95) \
                                                    + tf.multiply(e_params[i], 1 - 0.95)))
                
        
        
        
    
    def act(self, obs_, step, epsilon, noisy_):
     
        if step < self.memory_size : 
            action = np.random.randint(0, act_dim)
            return action
        
        obs_ = np.reshape(obs_, (-1, self.shape[0], self.shape[1], self.shape[2]))
        actions_values = self.sess.run(self.model_Q.prediction, feed_dict={self.states: obs_, self.batch_ : 1
                                                                          })
        
        if noisy_ == False : 
            random_action = np.random.randint(0, act_dim)
            if np.random.uniform() < epsilon:
                action = random_action 
            else : action =  np.argmax(actions_values, axis=1)[0]
          
        elif noisy_ == True and epsilon >= 0.01 : 
            delta = -np.log(1 - epsilon + epsilon/
                        env.action_space.n)
            q_values_adaptive = self.sess.run(self.m_adapt.prediction, feed_dict={self.states: obs_, self.batch_ : 1})  
            q_values_perturbed = self.sess.run(self.m_perturbed.prediction, feed_dict={self.states: obs_, self.batch_ : 1
                                                                              })  
            action = self.sess.run(self.action_.act_, feed_dict={self.obs:obs_,
                                                            self.step:step,           
                                                            self.delta:delta,
                                                            self.q_value:actions_values,
                                                            self.q_values_adaptive:q_values_adaptive,
                                                            self.q_values_perturbed:q_values_perturbed,
                                                            })
        
        else : 
            action =  np.argmax(actions_values, axis=1)[0]
        
        return action 

    def build_graph(self): 
        self._build_graph()
    
    
    @property
    def get_sess(self):
        return self.sess
    
    def train(self,  num_episodes, history_size, batch_size, epsilon = 1, render=False, noisy = False):
        
        
        logging.info('\nTraining starts\n')
        total_steps = 0
        
        r = []

        for episode in range(num_episodes):
            observation = env.reset()
            observation = process_observation(observation)
            observation = process_state_batch(observation)
            rewards = 0
            
            recent_history = []
            while True:
               
                if render:
                    env.render()
                
               
                recent_history.append(observation)
                
                if len(recent_history) == history_size + 1:
                    action = self.act(recent_history, total_steps, epsilon, noisy)
                    recent_history.pop(0)
                    
                else : 
                  a = np.zeros_like(observation) 
                  for i in range(history_size +1 - len(recent_history)) :
                    recent_history.insert(0, a)
                    
                  action = self.act(recent_history, total_steps, epsilon, noisy)
                  recent_history.pop(0)
                                      
                next_state, reward, done, info = env.step(action)
                next_state = process_observation(next_state)
                next_state = process_state_batch(next_state)
                #reward = process_reward(reward)
                
                df = np.hstack((observation.reshape(84*84,), action, reward, done, next_state.reshape(84*84,)))
                self.memory.store(df)
                
                if done:
                    print('\nEnd of episode {}, rewards last episode : {:.2f} '.format(episode, rewards))
                    r.append(rewards)
                    break

                observation = next_state
                rewards += reward
                total_steps += 1
               
                    
                if total_steps > self.memory_size and total_steps % 20 == 0  :
                   
                   self.learn(batch_size=batch_size)
                   
                if total_steps % 1000 == 0 and total_steps > self.memory_size:  
                    logging.info('Epsilon {:.3f}'.format(epsilon))
                    epsilon -= 0.0065
                    print("Epsilon", epsilon)
                    epsilon = max(0.01, epsilon)
                if total_steps % 3000 == 0 and total_steps >  self.memory_size:
                    self.update_target_weights('Target', 'Q_value')
                    print('target params replaced')
                if total_steps % 300000 == 0 : 
                    save_variables('v')
                
                
                
            
        if render:
            env.close()
        return r, epsilon





    def learn(self, batch_size=32):
        
        tree_idx, batch_memory, up_weights_ = self.memory.sample(batch_size)
        
        
        df = np.hstack(batch_memory)
        df = np.squeeze(df, axis = 0)
        
        states = df[:,:84*84]
        rewards = df[:, 84*84].reshape(-1,self.history_size + 1)[:,-1]
        actions = df[:, 84*84 +1].reshape(-1,self.history_size + 1)[:,-1]
        states_next = df[:,84*84 + 3:]
        
        states = np.reshape(states, (-1,84,84,1))
        states_next = np.reshape(states_next, (-1,84,84,1))
        self.merged_summary_op = tf.summary.merge_all()
        logs_path = os.path.expanduser('tb')
        self.summary_writer = tf.summary.FileWriter(logs_path,
                                            graph=tf.get_default_graph(),max_queue=20,
                                            flush_secs=20)
        
        q_tar_= self.sess.run(self.model_target.prediction, feed_dict={
                            self.states_next: states_next, self.batch_: batch_size})
        
        summary, actions_next_value = self.sess.run([self.merged_summary_op,self.model_Q.prediction], feed_dict={self.states: states_next, self.states_next: states_next,   self.batch_ :batch_size})
        self.summary_writer.add_summary(summary)   
        action_next = np.argmax(actions_next_value[:], axis =1)[0]
        
        
        _, cost, abs_errors = self.sess.run([self.model_Q.optimize, self.model_Q.error,
                                         self.model_Q.abs_error],
                                        feed_dict={self.states: states,
                                                   self.rewards: rewards,
                                                   self.actions: actions,
                                                   self.actions_next : action_next, 
                                                   self.up_weights: up_weights_,
                                                   self.q_tar: q_tar_,
                                                   self.batch_: batch_size})

        
        
        self.memory.batch_update(tree_idx, abs_errors)
        
        



In [0]:
import os
import tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
tf.logging.set_verbosity(tf.logging.WARN)
if __name__ == '__main__':
    import gym
    
    env = gym.make('MsPacman-v0')
    if tf.get_default_session() != None : 
           tf.get_default_session().close()
           tf.reset_default_graph() 
    try :              
        rl_model = RDeepRL(env, 'lstm')
        rl_model.build_graph()
        r_ = rl_model.train(100000,batch_size=128, history_size=2, noisy = True)
        
        save_variables('v_final')
    except KeyboardInterrupt:
        tf.get_default_session().close()
        print('Train() Interrupted')
        rl_model.get_sess.close()
        env.close()


[None, 9, 9, 64]
[None, 9, 9, 64]
[None, 9, 9, 64]
[None, 9, 9, 64]

End of episode 0, rewards last episode : 220.00 

End of episode 1, rewards last episode : 300.00 

End of episode 2, rewards last episode : 140.00 

End of episode 3, rewards last episode : 320.00 

End of episode 4, rewards last episode : 110.00 

End of episode 5, rewards last episode : 260.00 

End of episode 6, rewards last episode : 200.00 

End of episode 7, rewards last episode : 230.00 

End of episode 8, rewards last episode : 400.00 
Epsilon 0.9935
target params replaced

End of episode 9, rewards last episode : 2080.00 
Epsilon 0.9870000000000001

End of episode 10, rewards last episode : 320.00 
Epsilon 0.9805000000000001

End of episode 11, rewards last episode : 510.00 

End of episode 12, rewards last episode : 250.00 
Epsilon 0.9740000000000002
target params replaced

End of episode 13, rewards last episode : 320.00 

End of episode 14, rewards last episode : 520.00 
Epsilon 0.9675000000000002

End of